# Reconstructing Light-Curves

This notebook uses observations from CSP DR3 to explore the use of interpolation to supliment light-curve fits.

#### Table of Contents:
1. <a href='#setup'>Setup</a>: Downloads the CSP DR3 data set and registers the CSP filters with SNCosmo
1. <a href='#regression_vs_modeling'>Regression vs Modeling</a>: Compares fits to light-curve data using Salt2.4 and a Gaussian regression
1. <a href='#deconstructing'>Deconstructing and Reconstructing Light-Curves</a>: We remove every `n` points from a light-curve, and then try to re-create the missing data using interpolation and regression.
1. <a href='#chisq'>Tabulating Chi-Squared Values</a>: ...Unfinished...



In [ ]:
import sys

import numpy as np
import pandas as pd
import sncosmo
import sndata
from IPython.display import clear_output
from astropy.table import Column, vstack
from matplotlib import pyplot as plt
from matplotlib.ticker import MultipleLocator
from sndata.csp import dr1, dr3

sys.path.insert(0, '../')
import lc_regression
from lc_regression import utils


## Setup <a id='setup'></a>

This notebook relies on the CSP DR3 data set. We start by downloading the data release and registering the CSP filters with the `sncosmo` package. We also instantiate a Salt2 model for use later on.

In [ ]:
dr1.download_module_data()
dr3.download_module_data()
dr3.register_filters(force=True)

# Register filters with sncosmo
dr3.download_module_data()
dr3.register_filters(force=True)

# A Salt2.4 model
salt2 = sncosmo.Model(sncosmo.get_source('salt2', version='2.4'))

print('The DR3 data set:')
dr3.load_table(1)[:10]

## Regression vs Modeling <a id='regression_vs_modeling'></a>

We fit a CSP light-curve using both a Gaussian regression and the Salt2.4 model. Plotting the two allows us to visually build an intuition for the behavior of both approaches. One thing worth noting is how poorly the regression process works when extrapolating (which is not surprising). 

In [ ]:
def create_empty_figure(bands):
    """Create an empty figure to plot light-curves on
    
    Args:
        bands (list): A list of band names
    
    Returns:
        The figure object
        An array of figure axes
    """
    
    columns = 2
    rows = int(np.ceil(len(bands) / 2))
    fig_shape = (rows, columns)
    fig_size = (3 * rows, 6 * columns)

    fig, axes = plt.subplots(*fig_shape, figsize=fig_size)
    for axis in axes[rows - 1]:
        axis.set_xlabel('Time')

    for axis in axes.flatten()[::2]:
        axis.set_ylabel('Flux')
        
    for axis, band in zip(axes.flatten(), bands):
        axis.xaxis.set_minor_locator(MultipleLocator(1))
        axis.set_title(band)
        
    return fig, axes


def scatter_lc(data, bands, flux_col='flux', axes=None, **kwargs):
    """Scater plot a light curve
    
    Args:
        data   (Table): The measured photometric data
        bands   (list): Band names in desired plotting order
        flux_col (str): The column from ``data`` to plot on the y-axis
        axes (ndarray): Plot on existing axes
        Any other arguments for plt.scatter
    
    Returns:
        The figure object
        An array of figure axes
    """
    
    if axes is None:
        fig, axes = create_empty_figure(bands)
        
    else:
        fig = plt.gcf()
        
    plot_data = zip(bands, axes.flatten())
    for band_name, axis in plot_data:
        band_data = data[data['band'] == band_name]
        axis.scatter(band_data['time'], band_data[flux_col], **kwargs)    
        
    return fig, axes
        

def plot_regression(data, time, bands, pred_flux, pred_unc, axes=None):
    """Plot regression results in flux space
    
    Plots data['flux'] and data['fluxerr'] vs time in each band.
    Plots pred_flux and pred_unc vs time in each band.
    
    Args:
        data            (Table): The measured photometric data
        time          (ndarray): Time values to plot flux for
        bands            (list): Band names in desired plotting order
        pred_flux (list[float]): Predicted flux from the regression
        pred_unc  (list[float]): Flux uncertainty from the regression
        axes         (array): Plot on existing axes
    
    Returns:
        The figure object
        An array of figure axes
    """

    if axes is None:
        fig, axes = create_empty_figure(bands)
        
    else:
        fig = plt.gcf()
    
    # Plot the lightcurve data
    plot_data = zip(bands, axes.flatten(), pred_flux, pred_unc)
    for band_name, axis, f, fe in plot_data:
        band_data = data[data['band'] == band_name]
        axis.errorbar(band_data['time'], band_data['flux'],
                      band_data['fluxerr'], linestyle='', label=None)
        
        axis.plot(time, f, linestyle='--', color='black', label='Regression')
        axis.fill_between(
            time,
            f - fe,
            f + fe,
            alpha=.3,
            color='grey',
            label='Regression Error'
        )

        axis.set_xlim(min(time), max(time))
        
    return fig, axes


def plot_sncosmo_flux(time, bands, fitted_model, axes=None, **kwargs):
    """Plot the predicted flux of an sncosmo in each band vs time
    
    Args:
        time       (ndarray): Time values to plot flux for
        bands         (list): Band names in desired plotting order
        fitted_model (Model): An SNCosmo model fitted to the data
        axes         (array): Plot on existing axes
        
    Return:
        The figure object
        An array of figure axes
    """

    if axes is None:
        fig, axes = create_empty_figure(bands)
        
    else:
        fig = plt.gcf()
    
    for band, axis in zip(bands, axes.flatten()):
        try:
            band_flux = fitted_model.bandflux(band, time, zp=sndata.get_zp(band), zpsys='AB')

        except ValueError as e:
            print( f'{fitted_model.source.name} {fitted_model.source.version}: {e}')
            pass

        else:
            axis.plot(time, band_flux, **kwargs)
            
    return fig, axes


In [ ]:
# Pick a CSP target to use in demo plots
demo_id = '2005kc'
demo_data = dr3.get_data_for_id(demo_id)

bands = ['csp_dr3_' + b for b in 'ugriBJH']

# Fit a gaussian regression
gp = lc_regression.fit_gaussian_process(demo_data)
time = np.arange(min(demo_data['time']) - 1, max(demo_data['time']) + 1)
gp_flux, gp_unc = lc_regression.predict_light_curve(gp, bands, time)

# Fit salt2
salt2.set(z=demo_data.meta['z'])
salt2.set(t0=utils.get_csp_t0(demo_id))
fit_result, fitted_model = sncosmo.fit_lc(demo_data, salt2, ['x0', 'x1', 'c'])

# Plot modeled flux
fig, axes = plot_regression(demo_data, time, bands, gp_flux, gp_unc)
plot_sncosmo_flux(time, bands, fitted_model, axes, label='Salt2.4')
scatter_lc(demo_data, bands, axes=axes)

handles, labels = axes[0, 0].get_legend_handles_labels()
by_label = dict(zip(labels, handles))
fig.legend(by_label.values(), by_label.keys(), framealpha=1)

plt.tight_layout()


## Deconstructing and Reconstructing Light-Curves <a id='deconstructing'></a>

Now that we can regress light-curves, we see how well we can reconstruct missing photometric measurements. To do this we drop every `nth` value from a light-curve, estimate the missing values using either a linear interpolation or a Gaussian regression, and then fit the recovered values using Salt2. 

We note that a problem with this approach is that we might drop the last data point in a a given filter. This would require us to extrapolate when recovering the value, which, as seen above, is generally unreliable. To avoid cases where one or more extrapolated values might throw off our fit significantly, we never drop the last point in any given filter.


In [ ]:
def drop_nth_point(data, n):
    """Drop the nth point in each band
    
    Args:
        data (Table): A table of photometric data with column ``band``
        n      (int): How often to drop an observation
        
    Returns:
        A Table with some data removed
    """
    
    band_tables = []
    for band_data in data.group_by('band').groups:
        indices = (np.arange(1, len(band_data) + 1) % n) != 0
        indices[-1] = True  # Never drop the last value
        band_tables.append(band_data[indices])

    return vstack(band_tables)

def reconstruct_light_curve(data, n):
    """Drop the nth point in each band and then estimate the missing values
    
    The missing values are determined using both a Gaussian regression and a linear
    interpolation.
    
    Args:
        data (Table): A table of photometric data with column ``band``
        n      (int): How often to drop an observation
        
    Returns:
        A Table that is the same as ``data`` but with the added columns: 
          - interp_flux
          - gauss_flux
          - gauss_unc
    """
    
    deconstructed = drop_nth_point(data, n)
    gp = lc_regression.fit_gaussian_process(deconstructed)
    
    band_tables = []
    for band in set(deconstructed['band']):
        orig_data = data[data['band'] == band]
        dec_data = deconstructed[deconstructed['band'] == band]
        interp_flux = np.interp(orig_data['time'], dec_data['time'], dec_data['flux'])
        gauss_flux, gauss_unc = lc_regression.predict_band_flux(gp, band, orig_data['time'])
        
        orig_data['interp_flux'] = interp_flux
        orig_data['gauss_flux'] = gauss_flux
        orig_data['gauss_unc'] = gauss_unc
        band_tables.append(orig_data)

    out_table = vstack(band_tables)
    out_table['rec'] = out_table['flux'] != out_table['interp_flux']
    return out_table
    

For demonstration purposes, we choose to drop every third data point. Note that this will result in some values being extrapolated at the end of the observed phase range. The `rec` columns indicate whether a given value was recovered using an interpolation technique (i.e., whether the value was an `nth` data point).

In [ ]:
reconstructed = reconstruct_light_curve(demo_data, 3)
dropped_data = reconstructed[reconstructed['rec']]
kept_data = reconstructed[~reconstructed['rec']]

reconstructed[:10]


In [ ]:
# Fit salt2
salt2.set(z=demo_data.meta['z'])
salt2.set(t0=utils.get_csp_t0(demo_id))

_, all_fitted_model = sncosmo.fit_lc(demo_data, salt2, ['x0', 'x1', 'c'])

temp = reconstructed.copy()
temp['flux'] = temp['interp_flux']
_, interp_fitted_model = sncosmo.fit_lc(temp, salt2, ['x0', 'x1', 'c'])

temp = reconstructed.copy()
temp['flux'] = temp['gauss_flux']
_, gauss_fitted_model = sncosmo.fit_lc(temp, salt2, ['x0', 'x1', 'c'])


In [ ]:
# Plot results using interpolation
fig, axes = scatter_lc(dropped_data, bands, 'interp_flux', label='Interp', color='C1')
plot_sncosmo_flux(time, bands, interp_fitted_model, axes=axes, label='Interp', color='C1')

# Plot results using regression
scatter_lc(dropped_data, bands, 'gauss_flux', axes=axes, label='Gauss', color='C2')
plot_sncosmo_flux(time, bands, gauss_fitted_model, axes, label='Gauss', color='C2')

# Plot results using all the measured data
scatter_lc(kept_data, bands, axes=axes, label='Kept', color='C0')
scatter_lc(dropped_data, bands, axes=axes, label='Dropped', facecolors='none', color='k')
plot_sncosmo_flux(time, bands, all_fitted_model, axes, linestyle='--', color='k', label='Salt2 All')

handles, labels = axes[0, 0].get_legend_handles_labels()
by_label = dict(zip(labels, handles))
fig.legend(by_label.values(), by_label.keys(), framealpha=1)

plt.tight_layout()


## Tabulating Chi-Squared Values <a id='chisq'></a>

We perform the process demonstrated above for each lightcurve in the DR3 sample and tabulate the chi-squared values.


In [ ]:
def fit_target(data, model, vparams, n):
    """"""
    
    reconstructed = reconstruct_light_curve(data, n)
    
    gauss_table = reconstructed.copy()
    gauss_table['flux'] = gauss_table['gauss_flux']
    
    interp_table = reconstructed.copy()
    interp_table['flux'] = gauss_table['interp_flux']
    
    # Fit The model
    all_result, all_fitted_model = sncosmo.fit_lc(data, model, vparams)
    interp_result, interp_fitted_model = sncosmo.fit_lc(interp_table, model, vparams)
    gauss_result, gauss_fitted_model = sncosmo.fit_lc(gauss_table, model, vparams)
    
    return all_result, interp_result, gauss_result


table_rows = []
for target_data in dr3.iter_data(filter_func=utils.filter_has_csp_data):
    obj_id = target_data.meta['obj_id']
    salt2.set(z=target_data.meta['z'])
    salt2.set(t0=utils.get_csp_t0(obj_id))
    fit_results = fit_target(target_data, salt2, ['x0', 'x1', 'c'], 3)
    
    rows = [[obj_id, result.chisq, result.ndof] for result in fit_results]
    table_rows.extend(rows)
    
fit_results = pd.DataFrame(table_rows, columns=['obj_id', 'chisq', 'ndof'])
fit_results['interp_type'] = (len(fit_results) // 3) * ['none', 'linear', 'gauss']
fit_results.set_index(['obj_id', 'interp_type'])
    